In [2]:
import pandas as pd
import numpy as np
import time
from tsfresh.feature_extraction import extract_features, EfficientFCParameters, MinimalFCParameters,ComprehensiveFCParameters
import math
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = 20,10
rcParams['font.size'] = 22

/home/palash/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [20]:
sampling_rate = 20
window_size = 2 * sampling_rate

In [15]:
# for extracting windows from the data frame
def extract_windows(data,window_size,step):
    m = data.shape[0]
    total_windows = int(1 + (m-window_size)/step)
    lb = 0
    ub = 0
    window_collection = data.iloc[:window_size,:].assign(markers=lambda x: np.repeat(0,window_size))
    for window in range(1,total_windows):
        lb = step * window
        ub = lb + window_size
        window_collection = window_collection.append(data.iloc[lb:ub,:].assign(markers=lambda x: np.repeat(window,window_size)))
    # handling remaining samples at the bottom of the dataset
    if ub<data.shape[0]:
        window_collection = window_collection.append(data.iloc[ub:m,:].assign(markers=lambda x: np.repeat(window+1,m-ub)))
    return window_collection

In [17]:
def rolling_average(df, features):
    df[features] = df[features].rolling(3).mean()
    df.dropna(how='any',inplace=True)
    return df

In [36]:
header = ['Timestamp','Ac-x','Ac-y','Ac-z','Gy-x','Gy-y','Gy-z','Activity']
features = ['Ac-x','Ac-y','Ac-z','Gy-x','Gy-y','Gy-z']
standing_data = pd.read_csv('Standing_1515954467.329419.csv')
standing_data.columns = header
walking_data = pd.read_csv('Walking_1515954999.8478549.csv')
walking_data.columns = header
squats_data = pd.read_csv('Squats_1515955470.654806.csv')
squats_data.columns = header
jumping_jack_data = pd.read_csv('Jumping_Jack_1515955890.306268.csv')
jumping_jack_data.columns = header
lying_data = pd.read_csv('Lying_1515956102.0983102.csv')
lying_data.columns = header

standing_data = rolling_average(standing_data,features)
walking_data = rolling_average(walking_data,features)
squats_data = rolling_average(squats_data,features)
jumping_jack_data = rolling_average(jumping_jack_data,features)
lying_data = rolling_average(lying_data,features)

standing_data = extract_windows(standing_data,window_size,int(window_size*0.75))
walking_data = extract_windows(walking_data,window_size,int(window_size*0.75))
squats_data = extract_windows(squats_data,window_size,int(window_size*0.75))
jumping_jack_data = extract_windows(jumping_jack_data,window_size,int(window_size*0.75))
lying_data = extract_windows(lying_data,window_size,int(window_size*0.75))

walking_data['markers'] = walking_data['markers'] + standing_data['markers'].iloc[-1]+1
squats_data['markers'] = squats_data['markers'] + walking_data['markers'].iloc[-1] +1
jumping_jack_data['markers'] = jumping_jack_data['markers'] + squats_data['markers'].iloc[-1] + 1
lying_data['markers'] = lying_data['markers'] + jumping_jack_data['markers'].iloc[-1] + 1

In [41]:
final_data = standing_data
final_data = final_data.append([walking_data,squats_data,jumping_jack_data,lying_data],ignore_index=True)

In [93]:
columns = ['Timestamp','Ac-x','Ac-y','Ac-z','Gy-x','Gy-y','Gy-z','markers']
fc_parameters = {
    'abs_energy':None,
    'maximum': None,
    'mean': None,
    'median': None,
    'minimum': None,
    'standard_deviation': None,
    'variance': None,
    'kurtosis': None,
    'skewness': None,
}
extracted_features = extract_features(final_data[columns],
                                      column_id="markers",
                                      column_sort="Timestamp",
                                      default_fc_parameters=fc_parameters,
                                      n_jobs=2)


Feature Extraction: 100%|██████████| 10/10 [00:02<00:00,  3.72it/s]


In [94]:
extracted_features.columns

Index(['Ac-x__abs_energy', 'Ac-x__kurtosis', 'Ac-x__maximum', 'Ac-x__mean',
       'Ac-x__median', 'Ac-x__minimum', 'Ac-x__skewness',
       'Ac-x__standard_deviation', 'Ac-x__variance', 'Ac-y__abs_energy',
       'Ac-y__kurtosis', 'Ac-y__maximum', 'Ac-y__mean', 'Ac-y__median',
       'Ac-y__minimum', 'Ac-y__skewness', 'Ac-y__standard_deviation',
       'Ac-y__variance', 'Ac-z__abs_energy', 'Ac-z__kurtosis', 'Ac-z__maximum',
       'Ac-z__mean', 'Ac-z__median', 'Ac-z__minimum', 'Ac-z__skewness',
       'Ac-z__standard_deviation', 'Ac-z__variance', 'Gy-x__abs_energy',
       'Gy-x__kurtosis', 'Gy-x__maximum', 'Gy-x__mean', 'Gy-x__median',
       'Gy-x__minimum', 'Gy-x__skewness', 'Gy-x__standard_deviation',
       'Gy-x__variance', 'Gy-y__abs_energy', 'Gy-y__kurtosis', 'Gy-y__maximum',
       'Gy-y__mean', 'Gy-y__median', 'Gy-y__minimum', 'Gy-y__skewness',
       'Gy-y__standard_deviation', 'Gy-y__variance', 'Gy-z__abs_energy',
       'Gy-z__kurtosis', 'Gy-z__maximum', 'Gy-z__mean', 'Gy

In [95]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

In [96]:
gr = final_data.groupby('markers')

In [131]:
final_data.groupby('Activity').markers.count()

Activity
Jumping_Jack     2185
Lying            7529
Squats           2264
Standing        11167
Walking          9134
Name: markers, dtype: int64

In [145]:
X_train, X_test, y_train, y_test = train_test_split(extracted_features, gr.Activity.describe()['top'], test_size=0.2, random_state=0)

In [146]:
rf_cfl = RandomForestClassifier(n_estimators=100,n_jobs=3)

In [147]:
rf_model = rf_cfl.fit(X_train,y_train)

In [148]:
svm_model = svm.SVC(random_state=0).fit(X_train, y_train)

In [149]:
svm_model.score(X_test,y_test)

0.76543209876543206

In [150]:
rf_model.score(X_test,y_test)

1.0

In [151]:
rf_model.predict(X_test)

array(['Jumping_Jack', 'Standing', 'Standing', 'Walking', 'Standing',
       'Standing', 'Walking', 'Walking', 'Standing', 'Lying', 'Lying',
       'Walking', 'Standing', 'Standing', 'Lying', 'Lying', 'Walking',
       'Standing', 'Standing', 'Standing', 'Lying', 'Walking', 'Standing',
       'Walking', 'Walking', 'Lying', 'Lying', 'Lying', 'Standing',
       'Jumping_Jack', 'Standing', 'Standing', 'Standing', 'Walking',
       'Walking', 'Walking', 'Standing', 'Lying', 'Walking',
       'Jumping_Jack', 'Walking', 'Lying', 'Standing', 'Squats', 'Lying',
       'Lying', 'Jumping_Jack', 'Lying', 'Standing', 'Standing', 'Walking',
       'Standing', 'Walking', 'Walking', 'Walking', 'Standing', 'Walking',
       'Jumping_Jack', 'Jumping_Jack', 'Squats', 'Lying', 'Standing',
       'Walking', 'Walking', 'Walking', 'Lying', 'Lying', 'Lying',
       'Standing', 'Lying', 'Standing', 'Walking', 'Lying', 'Lying',
       'Lying', 'Standing', 'Lying', 'Walking', 'Walking', 'Standing',
       'Stan

In [152]:
for feature, importance in zip(extracted_features.columns,rf_model.feature_importances_):
    print(feature, importance)

Ac-x__abs_energy 0.0218065764368
Ac-x__kurtosis 0.00124256009566
Ac-x__maximum 0.0053142459622
Ac-x__mean 0.00182387444744
Ac-x__median 0.00118178123466
Ac-x__minimum 0.00658447597047
Ac-x__skewness 0.000221231789051
Ac-x__standard_deviation 0.0147932780057
Ac-x__variance 0.015383101451
Ac-y__abs_energy 0.0594436150056
Ac-y__kurtosis 0.00190438858383
Ac-y__maximum 0.0608563024942
Ac-y__mean 0.0393905912886
Ac-y__median 0.0571905761733
Ac-y__minimum 0.126344644584
Ac-y__skewness 0.000870537566489
Ac-y__standard_deviation 0.0348378559527
Ac-y__variance 0.0377033265617
Ac-z__abs_energy 0.0219088136938
Ac-z__kurtosis 0.000429556740352
Ac-z__maximum 0.0168203581563
Ac-z__mean 0.0168953751767
Ac-z__median 0.0103708403065
Ac-z__minimum 0.0137322714367
Ac-z__skewness 0.000414446651943
Ac-z__standard_deviation 0.01873986375
Ac-z__variance 0.0130087524894
Gy-x__abs_energy 0.0296307512173
Gy-x__kurtosis 0.000251831541509
Gy-x__maximum 0.0102853275884
Gy-x__mean 0.000797010441286
Gy-x__median 0.00

In [153]:
from sklearn.model_selection import cross_val_score

In [154]:
scores = cross_val_score(rfclf, extracted_features, gr.Activity.describe()['top'], cv=10)

In [155]:
scores

array([ 0.86585366,  1.        ,  1.        ,  1.        ,  1.        ,
        0.97530864,  1.        ,  0.9875    ,  1.        ,  1.        ])

In [156]:
from sklearn.externals import joblib

In [157]:
joblib.dump(rf_model,'RandomForest')

['RandomForest']

In [173]:
import socket, traceback
import pandas as pd
import time
from sklearn.externals import joblib
from tsfresh.feature_extraction import extract_features, EfficientFCParameters, MinimalFCParameters,ComprehensiveFCParameters
from io import StringIO
from sklearn.ensemble import RandomForestClassifier

header = ['Timestamp','Ac-x','Ac-y','Ac-z','Gy-x','Gy-y','Gy-z','marker']
fc_parameters = {
    'abs_energy':None,
    'maximum': None,
    'mean': None,
    'median': None,
    'minimum': None,
    'standard_deviation': None,
    'variance': None,
    'kurtosis': None,
    'skewness': None,
}
rf_model = joblib.load('RandomForest')
host = '10.42.0.1'
port = 5555

s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
s.bind((host, port))
count = 40
while 1:
    row = ''
    for i in range(40):
        try:
            message, address = s.recvfrom(8192)
            row = row + bytes.decode(message)[:-2] + ',1\n'
        except (KeyboardInterrupt, SystemExit):
            #dataset.close()
            print('\nClosing.')
            raise
        except:
            traceback.print_exc()
    df = pd.read_csv(StringIO(row))
    df.columns = header
    df_proccessed = extract_features(df,fc_parameters,n_jobs=3,column_sort='Timestamp',column_id='marker',disable_progressbar=True)
    print(rf_model.predict(df_proccessed.as_matrix()))





Feature Extraction: 100%|██████████| 6/6 [00:00<00:00, 1407.25it/s]

['Standing']






Feature Extraction: 100%|██████████| 6/6 [00:00<00:00, 1941.96it/s]

['Standing']






Feature Extraction: 100%|██████████| 6/6 [00:00<00:00, 1101.01it/s]

...

['Lying']






Feature Extraction: 100%|██████████| 6/6 [00:00<00:00, 4481.09it/s]

['Lying']



Process ForkPoolWorker-297:

Traceback (most recent call last):

  File "/home/palash/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/home/palash/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/palash/anaconda3/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/palash/anaconda3/lib/python3.6/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/home/palash/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/palash/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/palash/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
KeyboardInterrupt


KeyboardInterrupt: 

In [159]:
l = ['sdf sdf fgdhdfg ','sdfgsdg sdfgs']

In [162]:
import sys
sys.version_info[0]

3